# 4D Examples
These are the 4D examples used in the paper. There be dragons here, a lot of this code can take hours/days to compute. However, one it has been run, the resulting evaluation code is quite fast. Again, setup the context:

In [1]:
import llvmlite.ir as ll
import llvmlite.binding as llvm
from ctypes import CFUNCTYPE, c_int, c_float
import ctypes

load('helpers.sage')
load('boxspline.sage')
load('lattice.sage')
load('splinespace.sage')
load('codegen.sage')
load('horner.sage')

# Setup the LLVM context
llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()


In [2]:
# Define the lattices we care about
D4 = matrix(ZZ,[
       [-1, 0, 0, 0],
       [ 1, 0,-1,-1],
       [ 0,-1, 0, 1],
       [ 0,-1, 1, 0]
    ])

D4s = matrix(ZZ,[
        (2,0,0,1),
        (0,2,0,1),
        (0,0,2,1),
        (0,0,0,1)
    ])

D4 = IntegerLattice(D4)
D4s = IntegerLattice(D4s)

Let's take a look at the coset structure of these lattices. The $\mathcal{D}_4^*$ lattice has structure similar to the BCC lattice, that is, it is comprised of two interleaved Cartesian cosets, scaled by 2, with one shifted by (1,1,...,1).

In [3]:
scale, shifts = D4s.coset_structure()

print("Cartesian coset scale:\n%s" % scale)
print("Coset shifts: %s" % shifts)

Cartesian coset scale:
[2 0 0 0]
[0 2 0 0]
[0 0 2 0]
[0 0 0 2]
Coset shifts: [(0, 0, 0, 0), (1, 1, 1, 1)]


The structure of the $\mathcal{D}_4$ lattice is more similar to the FCC lattice. It's also comprised of a set of cartesian lattices (scaled by 2) but contains 8 cosets (each at the center of the 2-facets of the 4d hyper cube)

In [4]:
scale, shifts = D4.coset_structure()

print("Cartesian coset scale:\n%s" % scale)
print("Coset shifts: %s" % shifts)

Cartesian coset scale:
[2 0 0 0]
[0 2 0 0]
[0 0 2 0]
[0 0 0 2]
Coset shifts: [(0, 0, 0, 0), (0, 0, 1, 1), (0, 1, 0, 1), (0, 1, 1, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0), (1, 1, 1, 1)]


The splines listed in the paper "Symmetric Box Splines on Root Lattices" would be appropriate here, but they contain quite a few direction vectors that make them somewhat impractical to use in this instance. Instead, I define the following spline, which is somewhat of a cut-down version of one of the splines in that paper --- it only has 8 direction vectors, but is symmetric and works on both the $\mathcal{D}_4$ and $\mathcal{D}_4^*$ lattices. First we try to load it from the cache, if that fails, then we'll compute and cache it.

In [5]:
try:
    bs = load('cache/bs-4dim-8dir')
except:
    bs = BoxSpline(
        [(1, 1, 1, 1),
         (1, 1, -1, 1),
         (1, -1, 1, 1),
         (1, -1, -1, 1),
         (-1, 1, 1, 1),
         (-1, 1, -1, 1),
         (-1, -1, 1, 1),
         (-1, -1, -1, 1)])
    # Computation of the explicit form is lazy, so 
    # we have to call this method to explicitly compute
    # the pp form before caching the spline
    _ = bs.get_pp_regions()
    save(bs, 'cache/bs-4dim-8dir')

Now that we have a valid spline (I didn't verify that it's valid, but it's relatively easy to do so) we can combine this with the above lattices) we start with the $\mathcal{D}_4^*$ lattice. We pass the ```cache_id``` parameter to the SplineSpace constructor in order to cache preliminary results during computation --- there's a good chance you'll run out of system memory while this next operation is in progress (even with 64GB of system memory, this happens to me); it seems as if there's a consistent memory leak in Sage. To avoid the leak, we reload intermediate results from the cache, which avoids the computations that leak memory. Not a perfect solution, but it does work. Again, expect this to take about a day or two to compute.

In [ ]:
d4s_ss = SplineSpace(bs, D4s, cache_id = "d4s_8dir")

Builing region of evaluation... done!
Loading symmetry from cache...Ok!


In [ ]:
ecg = EvalCodeGen(d4s_ss,  target_triple=llvm.get_default_triple())
l = ecg.llvm_generate()
with open('cache/d4s_8dir.ll','w') as output:
    output.write(l.encode())

Do the same on the $\mathcal{D}_4$ lattice. Again, expect this to take 1-2 days (also, may possibly crash during computation)

In [ ]:
d4_ss = SplineSpace(bs, D4, cache_id = "d4_8dir")

In [ ]:
ecg = EvalCodeGen(d4_ss,  target_triple=llvm.get_default_triple())
l = ecg.llvm_generate()
with open('cache/d4_8dir.ll','w') as output:
    output.write(l.encode())

And that's it, you should have the interpolation code in the cache directory. For completeness, below I've included the quasi-interpolants used in the paper.

In [ ]:
d4s_quasi = [((-8, 0, 0, 0), -1/96),
             ((0, -8, 0, 0), -1/96),
             ((0, 0, -8, 0), -1/96),
             ((0, 0, 0, -8), -1/96),
             ((0, 0, 0, 8), -1/96),
             ((0, 0, 8, 0), -1/96),
             ((0, 8, 0, 0), -1/96),
             ((8, 0, 0, 0), -1/96),
             ((-4, -4, 0, 0), 1/288),
             ((-4, 0, -4, 0), 1/288),
             ((-4, 0, 0, -4), 1/288),
             ((-4, 0, 0, 4), 1/288),
             ((-4, 0, 4, 0), 1/288),
             ((-4, 4, 0, 0), 1/288),
             ((0, -4, -4, 0), 1/288),
             ((0, -4, 0, -4), 1/288),
             ((0, -4, 0, 4), 1/288),
             ((0, -4, 4, 0), 1/288),
             ((0, 0, -4, -4), 1/288),
             ((0, 0, -4, 4), 1/288),
             ((0, 0, 4, -4), 1/288),
             ((0, 0, 4, 4), 1/288),
             ((0, 4, -4, 0), 1/288),
             ((0, 4, 0, -4), 1/288),
             ((0, 4, 0, 4), 1/288),
             ((0, 4, 4, 0), 1/288),
             ((4, -4, 0, 0), 1/288),
             ((4, 0, -4, 0), 1/288),
             ((4, 0, 0, -4), 1/288),
             ((4, 0, 0, 4), 1/288),
             ((4, 0, 4, 0), 1/288),
             ((4, 4, 0, 0), 1/288),
             ((0, 0, 0, 0), 1)]
             
d4_quasi =  [((0, 0, 0, 0), 7/3),
             ((-1, -1, 0, 0), -1/18),
             ((-1, 0, -1, 0), -1/18),
             ((-1, 0, 0, -1), -1/18),
             ((-1, 0, 0, 1), -1/18),
             ((-1, 0, 1, 0), -1/18),
             ((-1, 1, 0, 0), -1/18),
             ((0, -1, -1, 0), -1/18),
             ((0, -1, 0, -1), -1/18),
             ((0, -1, 0, 1), -1/18),
             ((0, -1, 1, 0), -1/18),
             ((0, 0, -1, -1), -1/18),
             ((0, 0, -1, 1), -1/18),
             ((0, 0, 1, -1), -1/18),
             ((0, 0, 1, 1), -1/18),
             ((0, 1, -1, 0), -1/18),
             ((0, 1, 0, -1), -1/18),
             ((0, 1, 0, 1), -1/18),
             ((0, 1, 1, 0), -1/18),
             ((1, -1, 0, 0), -1/18),
             ((1, 0, -1, 0), -1/18),
             ((1, 0, 0, -1), -1/18),
             ((1, 0, 0, 1), -1/18),
             ((1, 0, 1, 0), -1/18),
             ((1, 1, 0, 0), -1/18)]

c4_quasi = [((-1, -1, -1, -1), 1/1296),
             ((-1, -1, -1, 0), -1/162),
             ((-1, -1, -1, 1), 1/1296),
             ((-1, -1, 0, -1), -1/162),
             ((-1, -1, 0, 0), 4/81),
             ((-1, -1, 0, 1), -1/162),
             ((-1, -1, 1, -1), 1/1296),
             ((-1, -1, 1, 0), -1/162),
             ((-1, -1, 1, 1), 1/1296),
             ((-1, 0, -1, -1), -1/162),
             ((-1, 0, -1, 0), 4/81),
             ((-1, 0, -1, 1), -1/162),
             ((-1, 0, 0, -1), 4/81),
             ((-1, 0, 0, 0), -32/81),
             ((-1, 0, 0, 1), 4/81),
             ((-1, 0, 1, -1), -1/162),
             ((-1, 0, 1, 0), 4/81),
             ((-1, 0, 1, 1), -1/162),
             ((-1, 1, -1, -1), 1/1296),
             ((-1, 1, -1, 0), -1/162),
             ((-1, 1, -1, 1), 1/1296),
             ((-1, 1, 0, -1), -1/162),
             ((-1, 1, 0, 0), 4/81),
             ((-1, 1, 0, 1), -1/162),
             ((-1, 1, 1, -1), 1/1296),
             ((-1, 1, 1, 0), -1/162),
             ((-1, 1, 1, 1), 1/1296),
             ((0, -1, -1, -1), -1/162),
             ((0, -1, -1, 0), 4/81),
             ((0, -1, -1, 1), -1/162),
             ((0, -1, 0, -1), 4/81),
             ((0, -1, 0, 0), -32/81),
             ((0, -1, 0, 1), 4/81),
             ((0, -1, 1, -1), -1/162),
             ((0, -1, 1, 0), 4/81),
             ((0, -1, 1, 1), -1/162),
             ((0, 0, -1, -1), 4/81),
             ((0, 0, -1, 0), -32/81),
             ((0, 0, -1, 1), 4/81),
             ((0, 0, 0, -1), -32/81),
             ((0, 0, 0, 0), 256/81),
             ((0, 0, 0, 1), -32/81),
             ((0, 0, 1, -1), 4/81),
             ((0, 0, 1, 0), -32/81),
             ((0, 0, 1, 1), 4/81),
             ((0, 1, -1, -1), -1/162),
             ((0, 1, -1, 0), 4/81),
             ((0, 1, -1, 1), -1/162),
             ((0, 1, 0, -1), 4/81),
             ((0, 1, 0, 0), -32/81),
             ((0, 1, 0, 1), 4/81),
             ((0, 1, 1, -1), -1/162),
             ((0, 1, 1, 0), 4/81),
             ((0, 1, 1, 1), -1/162),
             ((1, -1, -1, -1), 1/1296),
             ((1, -1, -1, 0), -1/162),
             ((1, -1, -1, 1), 1/1296),
             ((1, -1, 0, -1), -1/162),
             ((1, -1, 0, 0), 4/81),
             ((1, -1, 0, 1), -1/162),
             ((1, -1, 1, -1), 1/1296),
             ((1, -1, 1, 0), -1/162),
             ((1, -1, 1, 1), 1/1296),
             ((1, 0, -1, -1), -1/162),
             ((1, 0, -1, 0), 4/81),
             ((1, 0, -1, 1), -1/162),
             ((1, 0, 0, -1), 4/81),
             ((1, 0, 0, 0), -32/81),
             ((1, 0, 0, 1), 4/81),
             ((1, 0, 1, -1), -1/162),
             ((1, 0, 1, 0), 4/81),
             ((1, 0, 1, 1), -1/162),
             ((1, 1, -1, -1), 1/1296),
             ((1, 1, -1, 0), -1/162),
             ((1, 1, -1, 1), 1/1296),
             ((1, 1, 0, -1), -1/162),
             ((1, 1, 0, 0), 4/81),
             ((1, 1, 0, 1), -1/162),
             ((1, 1, 1, -1), 1/1296),
             ((1, 1, 1, 0), -1/162),
             ((1, 1, 1, 1), 1/1296)
           ]